# Create a Local Ethernet (Layer 2) Network

This notebook shows how to use create an isolated local Ethernet and connect compute nodes to it.  

## Step 1:  Configure the Environment


In [1]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
os.environ['FABRIC_CREDMGR_HOST']='beta-2.fabric-testbed.net'
os.environ['FABRIC_ORCHESTRATOR_HOST']='beta-7.fabric-testbed.net'
os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_config/fabric_token.json'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='pruth_0031379841'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/.ssh/id_ecdsa_fabric_bastion'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/work/fabric_config/slice-private-key'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/work/fabric_config/slice-public-key'


os.environ['FABRIC_PROJECT_ID']='b9847fa1-13ef-49f9-9e07-ae6ad06cda3f'
# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Step 2: Import the FABLlib Library


In [2]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import fablib

## Step 3 (Optional): Query for Available Tesbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [3]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name      CPUs  Cores    RAM (G)    Disk (G)     Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -----------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
LBNL         6  192/192  1536/1536  60600/60600  381/381                 2/2                             2/2                            10/10               4/4               2/2
RENC         6  186/192  1514/1536  60570/60600  379/381                 2/2                             2/2                            10/10               4/4               2/2
UKY          6  190/192  1530/1536  60590/60600  381/381                 2/2                             2/2                            10/10               4/4               2/2


## Step 4: Create the Experiment Slice

The following creates two nodes with basic NICs connected to an isolated local Ethernet.  

Two nodes are created and one NIC component is added to each node.  This example uses components of model `NIC_Basic` which are SR-IOV Virtual Function on a 100 Gpbs Mellanox ConnectX-6 PCI device. The VF is accessed by the node via PCI passthrough. Other NIC models are listed below. When using dedicated PCI devices the whole physical device is allocated to one node and the device is accessed by the node using PCI passthrough. Calling the `get_interfaces()` method on a component will return a list of interfaces. Many dedicated NIC components may have more than one port.  Either port can be connected to the network.

Next, add an `l2network` to the slice and pass the list of interfaces you want connected to this Ethernet. If all interfaces in the list are located on the same site, the network will automatically be a local Ethernet.  By default, a node is put on a random site.  If you want to ensure that your nodes are all on the same site you can specify the name of the site in the `add_node` methode.  You can use the `fablib.get_random_site()` method to get a random site name that can be used for both nodes.

NIC component models options:
- NIC_Basic: 100 Gbps Mellanox ConnectX-6 SR-IOV VF (1 Port)
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports) 
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports) 

In [4]:
slice_name = 'MySlice4'

site = fablib.get_random_site()
print(f"Site: {site}")

node1_name = 'Node1'
node2_name = 'Node2'
network_name='net1'
node1_nic_name = 'nic1'
node2_nic_name = 'nic2'

Site: RENC


In [5]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Node1
    node1 = slice.add_node(name=node1_name, site=site)
    iface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]
    
    # Node2
    node2 = slice.add_node(name=node2_name, site=site)
    iface2 = node2.add_component(model='NIC_Basic', name=node2_nic_name).get_interfaces()[0]
        
    # Network
    net1 = slice.add_l2network(name=network_name, interfaces=[iface1, iface2])

    #Submit Slice Request
    slice.submit(wait=False)  
except Exception as e:
    print(f"Exception: {e}")

## Step 5: Observe the Slice's Attributes

### Print the slice 

In [7]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
    
    slice.wait(progress=True)
    slice.wait_ssh(progress=True)
except Exception as e:
    print(f"Exception: {e}")

-----------  ------------------------------------
Slice Name   MySlice4
Slice ID     30fff949-b381-4601-94f2-edf89e5a8506
Slice State  Configuring
Lease End    2022-05-18 21:01:32 +0000
-----------  ------------------------------------
Waiting for slice ....... Slice state: StableOK
Waiting for slice . Slice state: StableOK
Waiting for ssh in slice .. ssh successful


## Print the Node List

In [8]:
try:
    slice = fablib.get_slice(name=slice_name)

    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

ID                                    Name    Site    Host                          Cores    RAM    Disk  Image            Management IP    State    Error
------------------------------------  ------  ------  --------------------------  -------  -----  ------  ---------------  ---------------  -------  -------
36cbb165-35ac-4f0a-a83c-71756a801542  Node1   RENC    renc-w1.fabric-testbed.net        2      8      10  default_rocky_8  152.54.15.40     Active
c577de27-97cd-4193-a82d-c1b3d8e85d30  Node2   RENC    renc-w1.fabric-testbed.net        2      8      10  default_rocky_8  152.54.15.45     Active


## Print the Node Details

In [9]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

-----------------  ----------------------------------------------------------------------------------------------------------------------------
ID                 36cbb165-35ac-4f0a-a83c-71756a801542
Name               Node1
Cores              2
RAM                8
Disk               10
Image              default_rocky_8
Image Type         qcow2
Host               renc-w1.fabric-testbed.net
Site               RENC
Management IP      152.54.15.40
Reservation State  Active
Error Message
SSH Command        ssh -i /Users/pruth/work/fabric_config/slice-private-key -J pruth_0031379841@bastion-1.fabric-testbed.net rocky@152.54.15.40
-----------------  ----------------------------------------------------------------------------------------------------------------------------
-----------------  ----------------------------------------------------------------------------------------------------------------------------
ID                 c577de27-97cd-4193-a82d-c1b3d8e85d30
Name               No

## Print the Interfaces

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

## Step 6 (Optional): Configure IP Addresses

Some experiments use FABRIC layer 2 networks to enable deploying non-IP layer 3 networks.  If this describes your exepriment, your nodes and network are ready. You can now login to the nodes and deploy your experiemnt.

Most users will want to configure IP addresses on there new nodes.  FABlib provides some useful methods to help you configure basic IP addreses. 

### Pick a Subnet

Create subnet and list of available IP addresses. All object are Python IP managment objects. You can use either IPv4 or IPv6 subents and addresses.

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet = IPv4Network("192.168.1.0/24")
    available_ips = list(subnet)[1:]
except Exception as e:
    print(f"Exception: {e}")

### Configure Node1

Get the node and the interface you wish to configure.  You can use `node.get_interface` to get the interface that is connected to the specified network.  Then `pop` an IP address from the list of available IPs and call `iface.ip_addr_add` to set the IP and subnet.  

Optionally, use the `node.execute()` method to show the results of adding the IP address.

In [ ]:
try:
    node1 = slice.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network_name) 
    node1_addr = available_ips.pop(0)
    node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet)
    
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

### Configure Node2

Repeat the steps to add the next available IP to the second node.

In [ ]:
try:
    node2 = slice.get_node(name=node2_name)        
    node2_iface = node2.get_interface(network_name=network_name)  
    node2_addr = available_ips.pop(0)
    node2_iface.ip_addr_add(addr=node2_addr, subnet=subnet)
    
    stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_os_interface()}')
    print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

## Step 7: Run the Experiment

We will find the ping round trip time for this pair of sites.  Your experiment should be more interesting!


In [ ]:
try:
    node1 = slice.get_node(name=node1_name)        

    stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Exception: {e}")

## Step 8: Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")